# Implementação do paper ``Generative Adversarial Networks``

Esse paper introduziu um novo framework para deep learning, chamado GANs, que consiste de um gerador e um discriminador que jogam uma competição entre si com o objetivo de gerar imagens realisticas, ou seja, que o gerador consiga confundir o discriminador para que ele não saiba diferenciar uma imagem real de uma imagem gerada artificialmente.

O artigo mostra que a tecnica é eficiente para gerar imagens artificiais realistas de digitos, faces e cenas naturais.

*Paper disponivel em: https://arxiv.org/abs/1406.2661

In [4]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from PIL import Image
import cv2
from torchvision import datasets, transforms
from torch import nn, optim
import os

os.makedirs("./frames", exist_ok=True)
os.makedirs("./frames_test", exist_ok=True)

## 1. Download do Dataset

Para essa implementação, utilizaremos o dataset MNIST. Além disso aplicaremos uma transformação simples, normalizando-o com 0.5 na media e variancia.

A depender de seu hardware, será necessario diminuir o batch_size para poupar recursos no treinamento em GPU/CPU

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

trainset = datasets.MNIST('./data', download=True, train=True, transform=transform)
valset = datasets.MNIST('./data', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=128, shuffle=True)

## 2. Criação dos Modelos

Conforme descrito no paper de Ian Goodfellow, devemos criar dois modelos: discriminador e gerador.

* Gerador: O gerador é uma rede com camadas lineares, que receberá um vetor de tamanho (N, L), onde L é o tamanho do espaço latente e tera como saida um vetor (N, 28*28), representando uma imagem. Essa rede será treinada para mapear um espaço latente em uma imagem, com isso, torna-se possivel gerar imagens artificiais de um determinado conjunto apenas gerando um vetor latente aleatorio!

* Discriminador: Rede com camadas lineares, que, receberá uma imagem com as seguintes dimensões (N, 28*28) e terá como output um escalar entre 0 e 1. Essa rede será treinada para tentar diferenciar o que é uma image "fake" de uma imagem "real".


In [ ]:
latent_size = 128

In [5]:
class Descriminator(nn.Module):
    
    def __init__(self):
        super(Descriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(28*28, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )
    
    def forward(self, img):
        return self.model(img.view(img.size(0), -1))
        
        
class Generator(nn.Module):
    
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_size, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 28*28),
            nn.Tanh(),
        )
    
    def forward(self, z):
        image = self.model(z)
        return image.view(image.size(0), *(1, 28, 28))
        

In [ ]:
generator = Generator().cuda()
descriminator = Descriminator().cuda()

generator.train()
descriminator.train()

## 3. Otimizador, Função de Perda e Parametros de Treinamento

Para esse problema, vamos utilizar o otimizador ADAM com os parametros $lr=2\cdot 10^{-4}$, $\beta_1 = 0.5$ e  $\beta_2 = 0.999$

Para a função de perda, devido ao carater binario do discriminador (0 ou 1, "Real" ou "Fake"), utilizaremos a função de entropia cruzada binaria, disponibilizada pelo PyTorch via ``nn.BCELoss``.

In [ ]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr=2e-4, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(descriminator.parameters(), lr=2e-4, betas=(0.5, 0.999))

loss_1 = torch.nn.BCELoss()

Também definiremos que a rotina de treinamento executará 3000 epochs. Além disso, o algoritmo que estamos implementando, preve uma rotina de $K$ iterações de treinamento do discriminador antes do treinamento do gerador, portanto, para nosso caso utilizaremos $k=2$.

In [ ]:
epochs = 3000
K = 2

## 4. Rotina de Treinamento

Por fim, podemos implementar a rotina de treinamento. Conforme descrito no paper, a rotina de treinamento é um jogo de competição entre o gerador e o discriminador, em que, primeiramente será utilizado imagens geradas e imagens do dataset, as quais serão classificadas pelo descriminador entre "real" (imagem do dataset) e "fake" (imagem gerada pelo gerador) e passará pelo processo de backpropagation.

Após essa etapa, será utilizado um conjunto de imagens "fake" geradas pelo gerador, que serão discriminadas e por fim, o erro da função perda será calculado considerando que as imagens representem imagens "reais", com o intuito de melhorar o gerador para que as imagens falsas se aproximem das imagens reais.

In [20]:
z_test = torch.Tensor(np.random.normal(0, 1, (1, latent_size))).cuda()

for epoch in range(epochs):
    loss_g = 0
    loss_d = 0

    for k in range(K):
        optimizer_D.zero_grad()
        image, label = next(iter(trainloader))

        image = image.cuda()

        real_label = torch.ones((image.size(0), 1)).cuda()
        fake_label = torch.zeros((image.size(0), 1)).cuda()

        z = torch.Tensor(np.random.normal(0, 1, (image.size(0), latent_size))).cuda()
        with torch.no_grad():
            fake_imgs = generator(z)
        fake_image_descriminated_label = descriminator(fake_imgs)
        real_image_descriminated_label = descriminator(image.detach())

        real_loss = loss_1(real_image_descriminated_label, real_label)
        fake_loss = loss_1(fake_image_descriminated_label, fake_label)

        d_loss = (real_loss/2 + (-1)*fake_loss/2)
        d_loss.backward()
        optimizer_D.step()

        loss_d += d_loss/K


    optimizer_G.zero_grad()

    z = torch.Tensor(np.random.normal(0, 1, (image.size(0), latent_size))).cuda()
    fake_imgs = generator(z)
    
    fake_image_descriminated_label = descriminator(fake_imgs) 

    generator_loss = loss_1(fake_image_descriminated_label, real_label)

    loss_g += generator_loss.item()

    generator_loss.backward()
    optimizer_G.step()
        
        
    if epoch % 100 == 0:    
        print(f"[Epoch: {epoch}] Generator Loss: {loss_g}; Discriminator Loss: {loss_d}")
        fake_imgs = generator(z_test)
        backtorgb = cv2.cvtColor(fake_imgs.cpu().detach().numpy().reshape(28, 28, 1)*255,cv2.COLOR_GRAY2RGB)
        cv2.imwrite(f"frames/epoch_{epoch}.jpg", backtorgb)
    

[Epoch: 0] Generator Loss: 0.6733267307281494; Discriminator Loss: -0.033357009291648865
[Epoch: 100] Generator Loss: 0.0; Discriminator Loss: -50.0
[Epoch: 200] Generator Loss: 0.0; Discriminator Loss: -50.0
[Epoch: 300] Generator Loss: 0.0; Discriminator Loss: -50.0
[Epoch: 400] Generator Loss: 0.0; Discriminator Loss: -50.0
[Epoch: 500] Generator Loss: 0.0; Discriminator Loss: -50.0
[Epoch: 600] Generator Loss: 0.0; Discriminator Loss: -50.0
[Epoch: 700] Generator Loss: 0.0; Discriminator Loss: -50.0
[Epoch: 800] Generator Loss: 0.0; Discriminator Loss: -50.0
[Epoch: 900] Generator Loss: 0.0; Discriminator Loss: -50.0
[Epoch: 1000] Generator Loss: 0.0; Discriminator Loss: -50.0
[Epoch: 1100] Generator Loss: 0.0; Discriminator Loss: -50.0
[Epoch: 1200] Generator Loss: 0.0; Discriminator Loss: -50.0
[Epoch: 1300] Generator Loss: 0.0; Discriminator Loss: -50.0
[Epoch: 1400] Generator Loss: 0.0; Discriminator Loss: -50.0
[Epoch: 1500] Generator Loss: 0.0; Discriminator Loss: -50.0
[Epoc

## 5. Verificação do Resultado do Gerador

Após a rotina de treinamento, vamos gerar 10 espaços latentes aleatorios, gerar as imagens que eles representam e salvar para verificação posterior.

In [16]:
from matplotlib import pyplot as plt
import cv2

generator.eval()
descriminator.eval()


for i in range(10):
    z = torch.Tensor(np.random.normal(0, 1, (1, latent_size))).cuda()

    fake_imgs = generator(z)
    backtorgb = cv2.cvtColor(fake_imgs.cpu().detach().numpy().reshape(28, 28, 1)*255,cv2.COLOR_GRAY2RGB)
    cv2.imwrite(f"frames_test/i_{i}.jpg", backtorgb)
#     plt.imshow(backtorgb)